In [2]:
import pandas as pd
import string
import requests
from datetime import datetime, timedelta
from sqlalchemy import create_engine

# Json com a lista das palavras chave, e as categorias informadas no documento da Pfizer
lista_de_palavras = {'palavras_chave':
                     ['COVID AND (vacina OR antiviral)',
                         'COVID-19 AND (vacina OR antiviral)',
                         'Coronavírus AND (vacina OR antiviral)',
                         'antiviral AND (covid OR covid-19)',
                         'nirmatrelvir',
                         'ritonavir',
                         'Paxlovid',
                         'COVID-19 AND tratamento',
                         'COVID AND tratamento',    
                         '"vacina bivalente" AND (covid OR covid-19)',
                         'covid AND teste',
                         'Covid-19 AND teste',
                         '"campanha de vacinação" AND (covid OR covid-19)',
                         'vacinação AND (covid OR covid-19)',
                         'Comirnaty',
                         '"vacina atenuada" AND (covid OR covid-19)',
                         '"vacina inativada" AND (covid OR covid-19)',
                         '"Sintomas Coronavírus"',
                         '"Sitomas Covid-19"',
                         '"Sintomas Covid"',
                         '"Ministério da saúde" AND (covid OR covid-19)',
                         'SBIm AND (covid OR covid-19)',
                         '"Jornada de imunização" AND (covid OR covid-19)',
                         '"eficácia vacina" AND (covid OR covid-19)',
                         '"segurança vacina" AND (covid OR covid-19)',
                         '"vacina mRNA" AND(Comirnaty OR RNA mensageiro)',
                         'Coronavírus AND (covid OR covid-19)',
                         'hospitalização AND (covid OR covid-19)',
                         'imunização AND (covid OR covid-19)',
                         'pandemia AND (covid OR covid-19)',
                         'sequelas AND (covid OR covid-19)',
                         '"Covid longa" AND (covid OR covid-19)',
                         'imunossuprimido AND (covid OR covid-19)',
                         'imunocomprometido AND (covid OR covid-19)',
                         'diagnóstico AND (covid OR covid-19)'],
                     'categorias': {
                         'Cardiologia':
                         ['AVC',
                             'Derrame',
                             'Arritmia Cardíaca',
                             'Fibrilação Atrial',
                             'Trombose',
                             'TVP',
                             'TEV',
                             'Embolia Pulmonar',
                             'Insuficiência Cardíaca',
                             'Pressão Alta',
                             'Hipertensão'
                          ],
                        #  'Hematologia':
                        #  ['Leucemia',
                        #      'Leucemia Aguda',
                        #      'Mielóide',
                        #      'Linfóide',
                        #      'Mieloma Múltiplo',
                        #      'Leucemia Linfoblastica Aguda',
                        #      'Mieloide',
                        #      'Leucemia linfoblástica aguda',
                        #      'Leucemia Mieloide Aguda',
                        #      'Hemofilia',
                        #   ],
                        #  'SNC': ['Depressão',
                        #          'Ansiedade',
                        #          'Antidepressivos',
                        #          'Sindrome do pânico',
                        #          'TDM',
                        #          'Transtorno Depressivo Maior',
                        #          'Depressivo',
                        #          'Ansiosos',
                        #          'Saúde Mental',
                        #          'Doença Mental',
                        #          'Transtorno Mental',
                        #          'Tristeza',
                        #          'Suicídio',
                        #          'Obesidade',
                        #          'Burnout',
                        #          'TAG',
                        #          ],
                        #  'Oncologia': [
                        #      'Câncer',
                        #      'Carcinoma',
                        #      'Neoplasia',
                        #      'Tumor',
                        #      'Imunoterapia',
                        #      'Quimioterapia',
                        #      'Hormonioterapia',
                        #      'Terapia Alvo',
                        #      'Transplante',
                        #      'Medula Óssea',
                        #      'Quinase',
                        #      'Sobrevida',
                        #      'Monoterapia',
                        #      'Câncer de Mama',
                        #      'Câncer de Rim',
                        #      'Câncer de Bexiga',
                        #      'Neoplasia renal',
                        #      'Neoplasia do trato gastrointestinal',
                        #      'Tumores neuroendocrinos de pâncreas',
                        #      'Câncer de Pulmão',
                        #      'Neoplasia pulmonar',
                        #      'Câncer de Pele',
                        #      'Melanoma',
                        #      'Colorretal',
                        #      'Pierre Fabre',
                        #      'Carcinoma de pele'],
                        #  'Reumatologia': [
                        #      'Anticorpo Antidroga',
                        #      'Anticorpo Antidroga Neutralizante',
                        #      'Anticorpo Monoclonal',
                        #      'Artrite',
                        #      'Artrite Idiopática Juvenil',
                        #      'Artrite Idiopática Juvenil Poliarticular',
                        #      'Artrite Psoriásica',
                        #      'Artrite Psoriática',
                        #      'Artrite relacionada a Entesite',
                        #      'Artrite Reumatoide',
                        #      'Biológicos',
                        #      'Congresso Brasileiro de Reumatologia',
                        #      'Doenças imunomediadas',
                        #      'Doenças Inflamatórias',
                        #      'Dor articular',
                        #      'Espondilite Anquilosante',
                        #      'Espondiloartrite Axial não Radiográfica',
                        #      'Fator de Necrose tumoral',
                        #      'Herpes Zoster',
                        #      'Lombalgia',
                        #      'Necrose tumoral',
                        #      'Sociedade Brasileira de Reumatologia',
                        #      'Terapia biológica',
                        #      'Trombose',
                        #      'Tuberculose',

                        # ]

                     }}


# função de busca na API
def apisearch(query: string):
    # Definir os parâmetros de busca
    search_params = {        
        "q": query,
        "lang": "pt",
        "sort_by": "relevancy",
        "page_size": 100
    }

    # Obter as notícias nos últimos 24 horas
    end_date = datetime.utcnow()
    start_date = end_date - timedelta(days=1)
    search_params["from"] = start_date.strftime("%Y-%m-%d")
    search_params["to"] = end_date.strftime("%Y-%m-%d")

    # Adiciona a chave de API da Newscatcher
    # api_key = "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A>"
    headers = {"x-api-key": "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"}

    # Faz a requisição para a  API
    url = "https://api.newscatcherapi.com/v2/search"
    response = requests.get(url, headers=headers, params=search_params)

    return response

#Salva o dataframe na base de dados
def saveInDatabase(dataframe):
    from sqlalchemy import create_engine

    host1 = 'pfizerdb.cqxct8aqwlso.us-east-2.rds.amazonaws.com'
    port1 = '3306'
    user1 = 'pfizer'
    passw1 = 'P!zer2597!'
    database1 = 'pfizerdb'

    mydb1 = create_engine('mysql+pymysql://' + user1 + ':' + passw1 +
                          '@' + host1 + ':' + str(port1) + '/' + database1, echo=False)
    dataframe.to_sql(name='news_covid', con=mydb1,
                     if_exists='append', index=False)


In [3]:
#Ao final do processo esse dataframe estará com todos dos dados das pesquisas concatenados
df_final = pd.DataFrame()

#Neste bloco, para cada palavra-chave do json, é feito um loop nas categorias
#             e nas áreas de interesse
for i in lista_de_palavras['palavras_chave']:
    print('*** Palavra-chave:'+ i)
    for c in lista_de_palavras['categorias']:
        print('** Categoria: '+ c)
        for k in lista_de_palavras['categorias'][c]:
            str_de_busca = i+' ' + ' "' + k + '"'
            print(str_de_busca) 
            response = apisearch(str_de_busca)
            
            try:
             articles = response.json()["articles"]
             df = pd.DataFrame(articles, columns=[
                "_id", "title", "published_date", "link"])
             df['category'] = 'COVID'
             df['associated'] = c
             df['keyword'] = k

        # Imprimir o dataframe
             df_final = pd.concat([df_final, df], ignore_index=True)
            except:
             continue
#refatora o json, agrupando as notícias duplicadas e adicionando os campos 'keyword' em um único campo, separados por vírgula
df_refatorado = df_final.groupby(["_id", "title", "published_date", "link", "category", "associated"])['keyword'].agg(
    lambda x: ', '.join(x)).reset_index()

#renomeia as colunas
df_refatorado = df_refatorado.rename(
         columns={'published_date': 'published', '_id': 'id', 'keyword':'kw','category':'categoria'})
#df_refatorado.to_excel('teste.xlsx')

#saveInDatabase(df_refatorado)

*** Palavra-chave:COVID AND (vacina OR antiviral)
** Categoria: Cardiologia
COVID AND (vacina OR antiviral)  "AVC"
COVID AND (vacina OR antiviral)  "Derrame"
COVID AND (vacina OR antiviral)  "Arritmia Cardíaca"
COVID AND (vacina OR antiviral)  "Fibrilação Atrial"
COVID AND (vacina OR antiviral)  "Trombose"
COVID AND (vacina OR antiviral)  "TVP"
COVID AND (vacina OR antiviral)  "TEV"
COVID AND (vacina OR antiviral)  "Embolia Pulmonar"
COVID AND (vacina OR antiviral)  "Insuficiência Cardíaca"
COVID AND (vacina OR antiviral)  "Pressão Alta"
COVID AND (vacina OR antiviral)  "Hipertensão"
*** Palavra-chave:COVID-19 AND (vacina OR antiviral)
** Categoria: Cardiologia
COVID-19 AND (vacina OR antiviral)  "AVC"
COVID-19 AND (vacina OR antiviral)  "Derrame"
COVID-19 AND (vacina OR antiviral)  "Arritmia Cardíaca"
COVID-19 AND (vacina OR antiviral)  "Fibrilação Atrial"
COVID-19 AND (vacina OR antiviral)  "Trombose"
COVID-19 AND (vacina OR antiviral)  "TVP"
COVID-19 AND (vacina OR antiviral)  "TEV"

In [4]:
import requests
import pandas as pd
import requests
import time
import pandas as pd
import pymysql
from sqlalchemy import create_engine
#from serpapi import GoogleSearch


from datetime import datetime, timedelta
today = datetime.today()
yesterday = datetime.today() - timedelta(days=1)


df=pd.DataFrame()

Cardio=["AVC","Derrame AND (Cardiologia OR Cardio)","Arritmia Cardíaca","Fibrilação Atrial","Fibrilação Atrial","Trombose","TVP","TEV","Embolia Pulmonar",
        "Insuficiência Cardíaca","Cardiologia Pressão Alta", "Cardíaca Pressão Alta", "Cardio Pressão Alta"]

for i in Cardio:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Cardio_result=df
Cardio_result=Cardio_result.rename(columns={'company':'kw'})
Cardio_result['categoria'] = 'Cardiologia'





df=pd.DataFrame()

Dermato=["Adalimumabe","Alopecia Areata","Alto Custo AND (Dematologia OR Dermato)","Anticorpo Antidroga","Anticorpo Antidroga","Anticorpo Antidroga Neutralizante",
         "Artrite Psoriásica","Artrite Psoriática","baricitinibe","Biológicos AND (Dematologia OR Dermato)","Biossimilar AND (Dematologia OR Dermato)","Biossimilares AND (Dematologia OR Dermato)","Biossimilaridade",
         "Dermatite","Dermatite Atópica","Doenças imunomediadas","Doenças Inflamatórias","Etanercepte","inibidor da JAK","Psoríase",
         "Psoríase em Placas","secuquinumabe","Terapia biológica","Tofacitinibe","Tuberculose","upadacitinibe","Ustequinumabe","Vitiligo",
         "Dermatite Atópica","SBD","Hidradenite Supurativa","Dupilumabe","Dupixent","Rinvoq","Olumiant","Sociedade Brasileira de Dermatologia",
         "Paulo Criado","Roberto Takaoka","Valeria Aoki","Sociedade Brasileira de Alegia e Imunologia","ASBAI","Eczema",
         "Dermatite de contato","Uticária","Coceira AND (Dematologia OR Dermato)","Pele sensível","Prurido","Depressão","Absenteísmo AND (Dematologia OR Dermato)","AAD","EADV ",
         "terapia sistemica","JADE Pfizer","Abrocinitibe", "JAK AND (Dematologia OR Dermato) NOT (motors OR carro OR concessionária OR playstation OR jogo OR Daxter)",
         "JAK1 AND (Dematologia OR Dermato) NOT (motors OR carro OR concessionária OR playstation OR jogo OR Daxter)",
         "JAK2 AND (Dematologia OR Dermato) NOT (motors OR carro OR concessionária OR playstation OR jogo OR Daxter)",
         "JAK3 AND (Dematologia OR Dermato) NOT (motors OR carro OR concessionária OR playstation OR jogo OR Daxter)"]

for i in Dermato:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Dermato_result=df
Dermato_result=Dermato_result.rename(columns={'company':'kw'})
Dermato_result['categoria'] = 'Dermatologia'





df=pd.DataFrame()

Reumato=["abatacepte","ACR - Colégio Americano de Reumatologia","Adalimumabe","Alto Custo AND (Reumato OR Reumatologia)","Anticorpo Antidroga","Anticorpo Antidroga Neutralizante",
         "Anticorpo Monoclonal","Artrite","Artrite Idiopática Juvenil","Artrite Idiopática Juvenil Poliarticular","Artrite Psoriásica",
         "Artrite Psoriática","Artrite relacionada a Entesite","Artrite Reumatoide","baricitinibe","Biológicos AND (Reumato OR Reumatologia)","Biossimilar AND (Reumato OR Reumatologia)",
         "Biossimilares AND (Reumato OR Reumatologia)",
         "Biossimilaridade AND (Reumato OR Reumatologia)","certolizumabe","Congresso Brasileiro de Reumatologia","Cristiano Campanholo","Doenças imunomediadas",
         "Doenças Inflamatórias","Dor articular","Espondilite Anquilosante","Espondiloartrite Axial não Radiográfica","Etanercepte",
         "Liga Europeia contra o Reumatismo","EULAR","Fator de Necrose tumoral","Herpes Zoster","Infliximabe","inibidor da JAK AND (Reumato OR Reumatologia)",
         "Karina Bonfiglioli","Licia Mota","LME","Lombalgia","Necrose tumoral","Ricardo Xavier","rituximabe","Sebastião Radominski",
         "secuquinumabe","Sociedade Brasileira de Reumatologia","Terapia biológica","tocilizumabe","Tofacitinibe","Trombose","Tuberculose",
         "Ultra-som articular","upadacitinibe","Ustequinumabe","Uveíte","Uveíte Pediátrica","Valderilio Feijó Azevedo","Vedolizumabe",
         "upadacitinibe","Dupilumabe","Rinvoq","Olumiant"]

for i in Reumato:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Reumato_result=df
Reumato_result=Reumato_result.rename(columns={'company':'kw'})
Reumato_result['categoria'] = 'Reumatologia'




df=pd.DataFrame()

Gastro=["Adalimumabe","Anticorpo Antidroga","Anti-Neutralizing Drug Antibody","Anticorpo Monoclonal","Biológico AND (gastroenterologia OR gastro)","Biossimilar AND (gastroenterologia OR gastro)",
        "Biossimilares AND (gastroenterologia OR gastro)","Biossimilaridade AND (gastroenterologia OR gastro)","Cicatrização de mucosa","Colite","Colite Ulcerativa","Colonoscopia","Coloproctologia",
        "Crohn","Diarreia sintoma","DII","Doença de Crohn","Doença de Crohn Fistulizante","Doença Gastrintestinal",
        "Doença Inflamatória Intestinal","Filgotinibe","Gastroenterologia","Manifestações extraintestinais","PANCCO AND (gastroenterologia OR gastro)","Paulo Kotze",
        "Retocolite","Retocolite Ulcerativa","SEBRADII","Ultra-som intestinal","terapia biológica","Tofacitinibe","Trombose","Tuberculose",
        "UEGW","Doenças imunomediadas","Doenças inflamatórias","ECCO-IBD","Infliximabe","IOIBD","JAK AND (gastroenterologia OR gastro)","LME"]

for i in Gastro:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Gastro_result=df
Gastro_result=Gastro_result.rename(columns={'company':'kw'})
Gastro_result['categoria'] = 'Gastroenterologia'






df=pd.DataFrame()

Onco=["Oncologia","Onco","Onco-Hemato","Câncer","Cancerígeno","Cancerígena","Carcinoma","Neoplasia","Tumor","Imunoterapia","Quimioterapia",
      "Hormonioterapia","Terapia Alvo","Transplante","Medula Óssea","Metástase","Mestastático","Células Anormais","Nódulo","Primeira Linha câncer",
      "Segunda Linha câncer","Terceira Linha câncer","Estágio I câncer","Estágio II câncer","Estágio III câncer","Estágio IV câncer","INCA",
      "ICESP","Hospital do Amor","AC Camargo","Quinase","Sobrevida","Monoterapia","Sobrevida Livre de Progressão","SLP câncer"]

for i in Onco:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Onco_result=df
Onco_result=Onco_result.rename(columns={'company':'kw'})
Onco_result['categoria'] = 'Oncologia'




df=pd.DataFrame()

Hemato=["Leucemia","Leucemia Aguda","Mielóide","Linfóide","LMA câncer","LMC câncer","LLA câncer","LLC câncer","Mieloma Múltiplo",
        "Leucemia Linfoblastica Aguda","CD33 câncer","anti-CD33 câncer","ALFA-0701 câncer","CD19 câncer","CD22 câncer","INO-VATE câncer",
        "Mieloide","Leucemia linfoblástica aguda","inotuzumabe Ozogamicina","cromossomo philadelphia negativo","DRM câncer","Blinatumomabe",
        "Gentuzumabe","InO câncer","Blina câncer","GO câncer","LLA Ph+ câncer","TCTH câncer","CAR-T câncer","BMT câncer","Elranatamab",
        "Leucemia Mieloide Aguda","Bosutinibe","Dasatinibe","Nilotinibe","inotuzumabe","Hemofilia"]

for i in Hemato:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass
    
Hemato_result=df    
Hemato_result=Hemato_result.rename(columns={'company':'kw'})
Hemato_result['categoria'] = 'Hematologia'
    




df=pd.DataFrame()

Biossimilares=["Biossimilar","Biossimilaridade","Imunogenicidade","Intercambialidade AND (biossimilar OR biossimilaridade)","Biossimilar Substituição automática",
               "Biossimilaridade Substituição automática","Switch Biossimilar","Cross Switch Biossimilar",
               "Switch reverso Biossimilar","Múltiplos switches Biossimilar","Originador Biossimilar","Referência Biossimilar","DMARDS","Biossimilar Biológicos",
               "Rituximabe","Trastuzumabe","Bevacizumabe","Libbs","Sandoz","Amgen","Celltrion","SOBRAFO Biossimilar","FLAB Biossimilar",
               "Valderilio Feijó Azevedo Biossimilar","Terapia biológica","intercambiabilidade biossimilar","Infliximabe","Adalimumabe"]

for i in Biossimilares:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Biossimilares_result=df
Biossimilares_result=Biossimilares_result.rename(columns={'company':'kw'})
Biossimilares_result['categoria'] = 'Biossimilares'




df=pd.DataFrame()

OncologiaMama=["Câncer de Mama","Palbociclibe","Abemaciclibe","Ribociclibe","Examestano","Talazoparibe","CDK câncer mama","iCDK câncer mama",
               "Ciclina câncer mama","HER2 câncer mama","HR+ câncer mama","HR- câncer mama","HER2+ câncer mama","RE+ câncer mama",
               "RE- câncer mama","BRCA1 câncer mama","BRCA2 câncer mama","CDK 4/6 câncer mama","CMM câncer mama",
               "receptor de fator de crescimento epidérmico humano 2","Câncer de Mama Metastático ","Câncer de Mama Avançado",
               "Inibidor de Ciclina","PALOMA-1 câncer mama","PALOMA-2 câncer mama","PALOMA-3 câncer mama","IRIS câncer mama",
               "FLATIRON câncer mama","PALOMAGE câncer mama","Receptor de estrogênio","Receptor hormonal ","Ciclina D1","PARPi",
               "Inibidor de PARP","EMBRACA","TALAPRO-1 ","Câncer de mama localmente avançado","mutação germinativa no BRCA","Trastuzumabe"]

for i in OncologiaMama:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

OncologiaMama_result=df
OncologiaMama_result=OncologiaMama_result.rename(columns={'company':'kw'})
OncologiaMama_result['categoria'] = 'Oncologia(Mama)'





df=pd.DataFrame()

OncologiaRCC=["Câncer de Rim","Câncer de Bexiga","Neoplasia renal","Neoplasia do trato gastrointestinal","Tumores neuroendocrinos de pâncreas",
              "Imunoterapia","Urotelial","Carcinoma Urotelial","anti-VEGF","Adjuvância","Célula Renal","Carcinoma avançado","Junho Verde",
              "Pazopanibe","Avelumabe","Sunitinibe","Axitinibe","Oncoguia","Vencer o cancer","IVOC","Fernando Maluf câncer","Citocina",
              "receptores de VEGF","sorafenibe","ADONIS câncer tumor","Células renais metastático","terapia combinação","RCC avançado",
              "Gist câncer tumor","Pnet","Liposarcomas","TKI","VEGFR","PDGFRB","pembrolizumabe","Ipi + Nivo","Javelin Renal 101","PD-L1",
              "SBU câncer tumor","CCRm","TKI + IO","Keytruda","AXItinibe + IO"]

for i in OncologiaRCC:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

OncologiaRCC_result=df
OncologiaRCC_result=OncologiaRCC_result.rename(columns={'company':'kw'})
OncologiaRCC_result['categoria'] = 'Oncologia(RCC)'





df=pd.DataFrame()

OncologiaPulmao=["Câncer de Pulmão","Neoplasia pulmonar","ROS1","ALK câncer","Crizotinibe","Lorlatinibe","Dacomitinibe","Estudo Crown",
                 "Alectinibe","Brigatinbe","Ensartinibe","CPNPC","ALK-Positivo","ALK+ câncer","EML4-ALK","Inibidor de ALK","ALK TKI",
                 "Quinase do Linfoma Anaplásico","Oncogene alk","ROS1+","Fusões gene ROS1","rearranjo de ROS1","ROS1-Positivo",
                 "Profile 1014","Profile 1001","câncer de pulmão de não pequenas celulas","entrectinibe","rearranjo de ALK",
                 "fusão do gene ALK","oncogene ALK","CPNPC ALK+","câncer de pulmão metastático","câncer de pulmão estágio IV"]

for i in OncologiaPulmao:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

OncologiaPulmao_result=df
OncologiaPulmao_result=OncologiaPulmao_result.rename(columns={'company':'kw'})
OncologiaPulmao_result['categoria'] = 'Oncologia(Pulmão)'





df=pd.DataFrame()

OncologiaMelanoma=["Câncer de Pele","Melanoma","Colorretal","Pierre Fabre","binimetinibe","encorafenibe","Carcinoma de pele","epiderme",
                   "derme","Braftovi","Mektovi","UV câncer","BRAF câncer","MEK câncer","RAF câncer","NF1 câncer","NRAS câncer",
                   "HRAS câncer","KRAS câncer","GNAQ câncer","GNA11 câncer","cKIT câncer","MAPK câncer","V600 câncer","Cobimetinibe",
                   "Dacarbazine","Ipilimimabe","Vemurafenibe","Dabrafenibe","Trametinibe","epitelial"]

for i in OncologiaMelanoma:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

OncologiaMelanoma_result=df
OncologiaMelanoma_result=OncologiaMelanoma_result.rename(columns={'company':'kw'})
OncologiaMelanoma_result['categoria'] = 'Oncologia(Melanoma)'





df=pd.DataFrame()

Pfizer=["Pfizer","Pfaizer","Faizer","Pifizer"]

for i in Pfizer:
    #EXTRAIR METADADOS NOTICIAS 
    try:
        url = "https://api.newscatcherapi.com/v2/search"
        querystring = {"q":i,"lang":"pt","countries":"BR","from":yesterday,"to":today}

        headers = {
            'x-api-key': "BbJk1vWYhB6jK6BJAoR8NFXL-acL8pqF0jdIQbbxI9A"
            }
        
        #response = requests.request("GET", url, headers=headers, params=querystring)
        response = requests.get(url, headers=headers, params=querystring).json()

        
        if response['articles']!=[]:
            df2 = pd.DataFrame.from_dict(response['articles'])
        # create column 'company' in the first position (position, column name, value)
            
        else:
            df2=pd.DataFrame()
            df2[["id","title","link","published","sub_articles","source"]] = pd.DataFrame([["", "", "","","",""]])
            
            
        df2.insert(0,'company', i)
        df=pd.concat([df,df2], ignore_index=True, sort=False)
    
    
        #COLETAR TEXTO DA NOTICIA
        url = "https://extract-news.p.rapidapi.com/v0/article"
        headers = {
            'x-rapidapi-key': "88b5804da0mshaec086ad3147560p16ac64jsn608ec3c7f56c",
            'x-rapidapi-host': "google-news.p.rapidapi.com"
            }
    except:
        pass

Pfizer_result=df
Pfizer_result=Pfizer_result.rename(columns={'company':'kw'})
Pfizer_result['categoria'] = 'Pfizer'




finaldata=pd.concat([Biossimilares_result,Cardio_result], axis=0)
finaldata=pd.concat([finaldata,Dermato_result], axis=0)
finaldata=pd.concat([finaldata,Gastro_result], axis=0)
finaldata=pd.concat([finaldata,Hemato_result], axis=0)
finaldata=pd.concat([finaldata,Pfizer_result], axis=0)
finaldata=pd.concat([finaldata,Reumato_result], axis=0)
finaldata=pd.concat([finaldata,Onco_result], axis=0)
finaldata=pd.concat([finaldata,OncologiaMama_result], axis=0)
finaldata=pd.concat([finaldata,OncologiaRCC_result], axis=0)
finaldata=pd.concat([finaldata,OncologiaPulmao_result], axis=0)
finaldata=pd.concat([finaldata,OncologiaMelanoma_result], axis=0)




finaldata=finaldata.rename(columns={'published_date':'published','_id':'id'})
finaldata1 = finaldata[['kw','id','title','link','published','categoria']]


###
# def convert_date(s):
#     month_dict = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May':'05', 'Jun':'06',
#                   'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

#     try:
#         d = s.split(" ")[1]
#         m = month_dict[s.split(" ")[2]]
#         y = s.split(" ")[3]
#     except:
#         return ""

#     return f'{d}-{m}-{y}'

# finaldata1["published"] = finaldata1["published"].apply(convert_date)

import datetime
finaldata1['published'] = pd.to_datetime(finaldata1['published'], errors="coerce", dayfirst=True )



#
#INSERT NO DB
import pandas as pd
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine

host1='pfizerdb.cqxct8aqwlso.us-east-2.rds.amazonaws.com'
port1='3306'
user1='pfizer'
passw1='P!zer2597!'
database1='pfizerdb'

# mydb1 = create_engine('mysql+pymysql://' + user1 + ':' + passw1 + '@' + host1 + ':' + str(port1) + '/' + database1 , echo=False)
# finaldata1.to_sql(name='google_news_brasil', con=mydb1, if_exists = 'append', index=False)
# finaldata1.to_sql(name='google_news_brasil_assunto', con=mydb1, if_exists = 'append', index=False)
# finaldata1.to_sql(name='google_news_brasil_last', con=mydb1, if_exists = 'append', index=False)

C:\Users\flavi\AppData\Local\Temp\ipykernel_44968\3896698263.py:645: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  finaldata=pd.concat([finaldata,Dermato_result], axis=0)
C:\Users\flavi\AppData\Local\Temp\ipykernel_44968\3896698263.py:646: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  finaldata=pd.concat([finaldata,Gastro_result], axis=0)
C:\Users\flavi\AppData\Local\Temp\ipykernel_44968\3896698263.py:647: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  finaldata=pd.concat([finaldata,Hemato_result], axis=0)
C:\Users\flavi\AppData\Local\Temp\ipykernel_44968\3896698263.py:648: FutureWarning: In a future versio